In [ ]:
%pip install -q node2vec

In [ ]:
import pandas as pd
import numpy as np
from evaluation_functions import KFold
from node2vec import Node2Vec
from tqdm import tqdm
import networkx as nx

import contextlib
from IPython.utils import io

data = KFold(cluster=True)

In [ ]:
lr_data_shape = data.lr[0].shape
hr_data_shape = data.hr[0].shape

print("Lr Data Shape:", lr_data_shape)
print("Hr Data Shape:", hr_data_shape)

In [ ]:
matrix_size_lr = 160
matrix_size_hr = 268

data.preprocessing(matrix_size_lr, matrix_size_hr)

In [ ]:

def generate_node2vec_embeddings(adjacency_matrices, dimensions=160, walk_length=8, num_walks=10, workers=8):
    embeddings = []
    for i in tqdm(range(len(adjacency_matrices))):
        with io.capture_output() as _:
            graph = nx.from_numpy_matrix(adjacency_matrices[i])
            node2vec = Node2Vec(graph, dimensions=dimensions, walk_length=walk_length, num_walks=num_walks, workers=workers)
            model = node2vec.fit(window=10, min_count=1, batch_words=4)
            embeddings.append(model.wv.vectors)
    return np.array(embeddings)

for i in range(0,3):
    lr_embeddings = generate_node2vec_embeddings(data.lr[i])
    np.save(f'Cluster CV/lr_embeddings_fold_{i+1}.npy', lr_embeddings)